In [ ]:
pip install pandas

# {first_name} - Recruiter First Name
# {company} - Recruiter Company Name

In [ ]:
import smtplib
import pandas as pd
import time
import os
import random
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.utils import formataddr

# Constants
EMAIL_ADDRESS = "EMAIL_ADDRESS" #Your Email ID
EMAIL_PASSWORD = "APPS_PASSWORD"  # Gmail Less Secure App Password, Use without space
RESUME_PATH = "RESUME_FILE_NAME.pdf" #File Name Replace
RESUME_FILENAME = "RESUME_FILE_NAME.pdf" #Your preferred file name to send
SENT_LOG_FILE = "sent_emails_log.csv" #Don't disturb this file at any cost
RECIPIENTS_CSV = "recipients.csv" #Data to import

# Load recipients
df = pd.read_csv(RECIPIENTS_CSV, header=None, names=['first_name', 'company', 'email'])
if os.path.exists(SENT_LOG_FILE):
    sent_log_df = pd.read_csv(SENT_LOG_FILE)
    already_sent = set(sent_log_df['email'])
else:
    already_sent = set()
with open(RESUME_PATH, 'rb') as f:
    resume_data = f.read()

# Email content
def create_email_body(first_name, company):
    return f"""Dear {first_name},

I hope this message finds you well.

My name is Varunprakash Shanmugam, and I recently graduated from City University of Seattle with a Master's degree in Computer Science. I am writing to express my strong interest in potential opportunities at {company}, particularly in Cloud Engineering/DevOps/IT Support roles.

I have practical experience designing and implementing cloud infrastructure solutions. For instance, I architected multi-cloud solutions using CloudFormation and developed DevOps pipelines integrating EC2, S3, and Lambda services. My experience also includes architecting and deploying distributed systems using Kubernetes and implementing CI/CD pipelines with tools like GitLab CI and Jenkins. I'm proficient in technologies such as AWS, Azure, Docker, Kubernetes, Cloud Formation, and Python.

Thank you for your time and consideration. I look forward to the possibility of connecting.

Best regards,  
Varunprakash Shanmugam  
https://linkedin.com/in/varunprakashs
"""

#Danger, don't touch the below code except to change FROM Name and Subject Line.
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
except smtplib.SMTPAuthenticationError:
    print("Authentication failed. Please check your email credentials.")
    exit()
log = []
email_counter = 0
batch_size = random.randint(3, 7)
for index, row in df.iterrows():
    recipient_email = row['email']
    first_name = row['first_name']
    company = row['company']

    if recipient_email in already_sent:
        print(f"Skipped (already sent): {recipient_email}")
        continue

    msg = MIMEMultipart()
    msg['From'] = formataddr(("Varunprakash Shanmugam", EMAIL_ADDRESS))
    msg['To'] = recipient_email
    msg['Subject'] = f"Looking for Cloud Engineering / DevOps Opportunities at {company}"

    msg.add_header('X-Priority', '1')
    msg.add_header('X-MSMail-Priority', 'High')
    msg.add_header('Importance', 'High')

    body = create_email_body(first_name, company)
    msg.attach(MIMEText(body, 'plain'))

    part = MIMEApplication(resume_data, Name=RESUME_FILENAME)
    part['Content-Disposition'] = f'attachment; filename="{RESUME_FILENAME}"'
    msg.attach(part)

    try:
        server.sendmail(EMAIL_ADDRESS, recipient_email, msg.as_string())
        print(f"Email sent to {recipient_email}")
        # Immediately log success to CSV file
        pd.DataFrame([{"email": recipient_email}]).to_csv(
            SENT_LOG_FILE, mode='a', header=not os.path.exists(SENT_LOG_FILE), index=False
        )
        already_sent.add(recipient_email)
    except Exception as e:
        print(f"Failed to send email to {recipient_email}: {e}")

    # Per-email random delay
    delay = random.uniform(2, 12)
    print(f"Sleeping {delay:.2f}s before next email...")
    time.sleep(delay)

    # Batch pause logic
    email_counter += 1
    if email_counter >= batch_size:
        batch_delay = random.randint(30, 90)
        print(f"Batch limit reached. Sleeping {batch_delay}s to avoid spam detection...")
        time.sleep(batch_delay)
        email_counter = 0
        batch_size = random.randint(3, 7)

server.quit()
print("All emails processed. Log file updated after each successful send.")